In [34]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done
^C
^C


In [35]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [36]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Spark").getOrCreate()
spark

In [37]:
from pyspark.sql import functions
print(dir(functions))

['Column', 'DataFrame', 'DataType', 'PandasUDFType', 'PythonEvalType', 'SparkContext', 'StringType', 'UserDefinedFunction', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_create_column_from_literal', '_create_lambda', '_create_udf', '_get_get_jvm_function', '_get_lambda_parameters', '_invoke_binary_math_function', '_invoke_function', '_invoke_function_over_column', '_invoke_higher_order_function', '_options_to_str', '_test', '_to_java_column', '_to_seq', '_unresolved_named_lambda_variable', 'abs', 'acos', 'acosh', 'add_months', 'aggregate', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_contains', 'array_distinct', 'array_except', 'array_intersect', 'array_join', 'array_max', 'array_min', 'array_position', 'array_remove', 'array_repeat', 'array_sort', 'array_union', 'arrays_overlap', 'arrays_zip', 'asc', 'asc_nulls_first', 'asc_nulls_last', 'ascii', 'asin', 'asinh', 'assert_true', 'atan', 'atan2', 'atanh', 'a

In [38]:
from pyspark.sql.functions import lit,lower, upper,concat,substring

emp_info = spark.read.csv('Employee_info.csv', header=True, sep=",", inferSchema=True)
emp_info.show(10)

+------+--------+-----------+----+-------------+
|Emp_id|    Name|Emp_dept_id|Days|working_hours|
+------+--------+-----------+----+-------------+
|     1|   Smith|      45645|   8|            8|
|     2|    Rose|      46887|   9|            6|
|     3| Juliana|      45645|   7|            8|
|     4|Anjelina|      46578|   6|            6|
|     5|  Robert|      47863|   1|            8|
|     6|   Bruce|      43879|   2|            8|
|     7|   Brown|      48378|   3|            5|
|     8|    Bean|      45789|   1|            8|
|     9|    Nike|      42389|   4|            6|
|    10|    Nate|      41868|   5|            6|
+------+--------+-----------+----+-------------+



In [39]:
from pyspark.sql.functions import lit,lower, upper,concat,substring

emp = spark.read.csv('employee.csv', header=True, sep=",", inferSchema=True)
emp.show(10)

+---+------+----------+------+-----+
| ID|Salary|Department|  city|state|
+---+------+----------+------+-----+
|  1| 45000|        IT|   SFO|   CA|
|  2| 50000|        IT|Jersey|   MA|
|  3| 65000|       IOT|    NY|   DA|
|  4| 45000|        CS|    MC|   NJ|
|  5| 35000|        CS|   SAN|   DE|
|  6| 50000|       EEE|Jersey|   DE|
|  7| 30000|       EEE|JORGEA|   JD|
|  8| 40000|        SE|Newark|   PO|
|  9| 35000|        SE| TOKYO|   TA|
| 10| 40000|        CS|Newark|   BD|
+---+------+----------+------+-----+



In [45]:
emp_info.join(emp, emp_info.Emp_id == emp.ID, "cross").select(emp_info["Emp_id"], emp_info["Name"], emp["Salary"]).show()


+------+--------+------+
|Emp_id|    Name|Salary|
+------+--------+------+
|     1|   Smith| 45000|
|     2|    Rose| 50000|
|     3| Juliana| 65000|
|     4|Anjelina| 45000|
|     5|  Robert| 35000|
|     6|   Bruce| 50000|
|     7|   Brown| 30000|
|     8|    Bean| 40000|
|     9|    Nike| 35000|
|    10|    Nate| 40000|
+------+--------+------+



In [46]:
emp_info.join(emp, emp_info.Emp_id == emp.ID, "outer").select(emp_info["Emp_id"], emp_info["Name"], emp["Salary"]).show()

+------+--------+------+
|Emp_id|    Name|Salary|
+------+--------+------+
|     1|   Smith| 45000|
|     6|   Bruce| 50000|
|     3| Juliana| 65000|
|     5|  Robert| 35000|
|     9|    Nike| 35000|
|     4|Anjelina| 45000|
|     8|    Bean| 40000|
|     7|   Brown| 30000|
|    10|    Nate| 40000|
|     2|    Rose| 50000|
+------+--------+------+



In [47]:
emp_info.join(emp, emp_info.Emp_id == emp.ID, "leftouter").select(emp_info["Emp_id"], emp_info["Name"], emp["Salary"]).show()

+------+--------+------+
|Emp_id|    Name|Salary|
+------+--------+------+
|     1|   Smith| 45000|
|     2|    Rose| 50000|
|     3| Juliana| 65000|
|     4|Anjelina| 45000|
|     5|  Robert| 35000|
|     6|   Bruce| 50000|
|     7|   Brown| 30000|
|     8|    Bean| 40000|
|     9|    Nike| 35000|
|    10|    Nate| 40000|
+------+--------+------+



In [48]:
emp_info.join(emp, emp_info.Emp_id == emp.ID, "rightouter").select(emp_info["Emp_id"], emp_info["Name"], emp["Salary"]).show()

+------+--------+------+
|Emp_id|    Name|Salary|
+------+--------+------+
|     1|   Smith| 45000|
|     2|    Rose| 50000|
|     3| Juliana| 65000|
|     4|Anjelina| 45000|
|     5|  Robert| 35000|
|     6|   Bruce| 50000|
|     7|   Brown| 30000|
|     8|    Bean| 40000|
|     9|    Nike| 35000|
|    10|    Nate| 40000|
+------+--------+------+



In [49]:
emp_info.join(emp, emp_info.Emp_id == emp.ID, "fullouter").select(emp_info["Emp_id"], emp_info["Name"], emp["Salary"]).show()

+------+--------+------+
|Emp_id|    Name|Salary|
+------+--------+------+
|     1|   Smith| 45000|
|     6|   Bruce| 50000|
|     3| Juliana| 65000|
|     5|  Robert| 35000|
|     9|    Nike| 35000|
|     4|Anjelina| 45000|
|     8|    Bean| 40000|
|     7|   Brown| 30000|
|    10|    Nate| 40000|
|     2|    Rose| 50000|
+------+--------+------+

